This notebook prepares the documentation on Newton's method, deflation, and multiplicity.

# Newton's method, Deflation and Multiplicity

In [1]:
from phcpy.solutions import make_solution, strsol2dict
from phcpy.deflation import double_newton_step
from phcpy.deflation import double_double_newton_step
from phcpy.deflation import quad_double_newton_step
from phcpy.deflation import double_deflate
from phcpy.deflation import double_multiplicity

PHCv2.4.88 released 2023-12-26 works!


## 1. Newton's method

In [2]:
pols = ['x^2 + 4*y^2 - 4;', '2*y^2 - x;']

At a regular solution, Newton's method doubles the accuracy in each step. 

In [3]:
sols = [make_solution(['x', 'y'], [1.23, -0.786])]
print(sols[0])

t : 0.000000000000000E+00 0.000000000000000E+00
m : 1
the solution for t :
 x : 1.230000000000000E+00  0.0
 y : -7.860000000000000E-01  0.0
== err : 0.000E+00 = rco : 1.000E+00 = res : 0.000E+00 =


In [4]:
sols = double_newton_step(pols, sols)
print(sols[0])

t :  0.00000000000000E+00   0.00000000000000E+00
m : 0
the solution for t :
 x :  1.23607623318386E+00   0.00000000000000E+00
 y : -7.86154018188250E-01   0.00000000000000E+00
== err :  6.230E-03 = rco :  1.998E-01 = res :  3.706E-05 =


Observe the values for ``err`` (forward error), ``rco``(estimated inverse of the condition number), and ``res`` (the residual or backward error).  

The multiplicity field ``m`` turned ``0`` because the default tolerance was not reached and the solution could not be counted as a proper solution.  Let us reset the multiplicity, taking the values from the ``sols[0]``.

In [5]:
sold = strsol2dict(sols[0])
sold

{'t': 0j,
 'm': 0,
 'err': 0.00623,
 'rco': 0.1998,
 'res': 3.706e-05,
 'x': (1.23607623318386+0j),
 'y': (-0.78615401818825+0j)}

In [6]:
sol = make_solution(['x', 'y'], [sold['x'], sold['y']])
print(sol)

t : 0.000000000000000E+00 0.000000000000000E+00
m : 1
the solution for t :
 x : 1.236076233183860E+00  0.000000000000000E+00
 y : -7.861540181882500E-01  0.000000000000000E+00
== err : 0.000E+00 = rco : 1.000E+00 = res : 0.000E+00 =


In [7]:
sols = [sol]
sols = double_newton_step(pols, sols)
print(sols[0])

t :  0.00000000000000E+00   0.00000000000000E+00
m : 0
the solution for t :
 x :  1.23606797751503E+00   0.00000000000000E+00
 y : -7.86151377766704E-01   0.00000000000000E+00
== err :  1.090E-05 = rco :  1.998E-01 = res :  1.100E-10 =


Observe that the value of ``res`` dropped from magnitude ``1.0e-5`` down to ``1.0e-10``, corresponding to the well conditioning of the root.  However, the multiplicy field is still zero because the estimate for the forward error is still too high.

In [8]:
sold = strsol2dict(sols[0])
sol = make_solution(['x', 'y'], [sold['x'], sold['y']])
sols = [sol]
sols = double_newton_step(pols, sols)
print(sols[0])

t :  0.00000000000000E+00   0.00000000000000E+00
m : 1
the solution for t :
 x :  1.23606797749979E+00   0.00000000000000E+00
 y : -7.86151377757423E-01   0.00000000000000E+00
== err :  2.452E-11 = rco :  1.998E-01 = res :  4.441E-16 =


The value for the residual ``res`` is very close to machine precision and the solution is considered a proper regular solution.

We can double the precision to double double.

In [9]:
sols = double_double_newton_step(pols, sols)
print(sols[0])

t : 0.00000000000000000000000000000000E+00      0.00000000000000000000000000000000E+00    
m : 1
the solution for t :
 x : 1.23606797749978969640917366873130E+00      0.00000000000000000000000000000000E+00    
 y : -7.86151377757423286069558585843026E-01     0.00000000000000000000000000000000E+00    
== err :  3.036E-16 = rco :  1.998E-01 = res :  3.944E-31 =


In [10]:
sols = double_double_newton_step(pols, sols)
print(sols[0])

t : 0.00000000000000000000000000000000E+00      0.00000000000000000000000000000000E+00    
m : 1
the solution for t :
 x : 1.23606797749978969640917366873128E+00      0.00000000000000000000000000000000E+00    
 y : -7.86151377757423286069558585842966E-01     0.00000000000000000000000000000000E+00    
== err :  6.272E-32 = rco :  1.998E-01 = res :  0.000E+00 =


and then to quad double precision:

In [11]:
sols = quad_double_newton_step(pols, sols)
print(sols[0])

t : 0.0000000000000000000000000000000000000000000000000000000000000000E+00      0.0000000000000000000000000000000000000000000000000000000000000000E+00    
m : 1
the solution for t :
 x : 1.2360679774997896964091736687312762354406183596115257242708972454E+00      0.0000000000000000000000000000000000000000000000000000000000000000E+00    
 y : -7.8615137775742328606955858584295892952312205783772323766490197015E-01     0.0000000000000000000000000000000000000000000000000000000000000000E+00    
== err :  7.070E-33 = rco :  1.998E-01 = res :  2.101E-64 =


## 2. Deflation

At an isolated singular solution, *deflation* is a method to restore the quadratic convergence of Newton's method.

In [12]:
pols = ['(29/16)*x^3 - 2*x*y;', 'x^2 - y;']

In [13]:
sols = [make_solution(['x', 'y'],[1.0e-6, 1.0e-6])]
print(sols[0])

t : 0.000000000000000E+00 0.000000000000000E+00
m : 1
the solution for t :
 x : 1.000000000000000E-06  0.0
 y : 1.000000000000000E-06  0.0
== err : 0.000E+00 = rco : 1.000E+00 = res : 0.000E+00 =


In [14]:
sols = double_newton_step(pols, sols)
print(sols[0])

t :  0.00000000000000E+00   0.00000000000000E+00
m : 0
the solution for t :
 x :  9.99999906191101E-07   0.00000000000000E+00
 y :  9.99999812409806E-13   0.00000000000000E+00
== err :  1.000E-06 = rco :  5.625E-13 = res :  1.875E-19 =


In [15]:
sols = double_newton_step(pols, sols)
print(sols[0])

t :  0.00000000000000E+00   0.00000000000000E+00
m : 0
the solution for t :
 x :  6.66666604160106E-07   0.00000000000000E+00
 y :  3.33333270859482E-13   0.00000000000000E+00
== err :  3.333E-07 = rco :  2.778E-14 = res :  1.111E-13 =


In [16]:
solsd = double_deflate(pols, sols)
print(solsd[0])

t :  0.00000000000000E+00   0.00000000000000E+00
m : 1
the solution for t :
 x :  9.46532112069346E-24   4.09228221015004E-24
 y :  1.02357542351685E-24  -2.03442589046821E-24
== err :  5.292E-12 = rco :  5.608E-03 = res :  1.885E-15 =


Deflation also works on systems with more equations than unknowns.

In [17]:
pols = ['x^2;', 'x*y;', 'y^2;']

In [18]:
sols = [make_solution(['x', 'y'], [1.0e-6, 1.0e-6])]
print(sols[0])

t : 0.000000000000000E+00 0.000000000000000E+00
m : 1
the solution for t :
 x : 1.000000000000000E-06  0.0
 y : 1.000000000000000E-06  0.0
== err : 0.000E+00 = rco : 1.000E+00 = res : 0.000E+00 =


In [19]:
sols = double_deflate(pols, sols, tolrnk=1.0e-8)
print(sols[0])

t :  0.00000000000000E+00   0.00000000000000E+00
m : 1
the solution for t :
 x :  1.25000000000000E-07   0.00000000000000E+00
 y :  1.25000000000000E-07   0.00000000000000E+00
== err :  1.250E-07 = rco :  8.165E-01 = res :  1.562E-14 =


In [20]:
sols = double_deflate(pols, sols, tolrnk=1.0e-4)
print(sols[0])

t :  0.00000000000000E+00   0.00000000000000E+00
m : 1
the solution for t :
 x : -5.87747175411144E-39  -5.14278778484751E-39
 y :  2.93873587705572E-39  -1.83670992315982E-40
== err :  2.757E-23 = rco :  4.082E-01 = res :  1.984E-38 =


## 3. multiplicity structure

The multiplicity can be computed *locally* starting at the solution.

In [21]:
pols = [ 'x**2+y-3;', 'x+0.125*y**2-1.5;']

In [22]:
sol = make_solution(['x', 'y'], [1, 2])
print(sol)

t : 0.000000000000000E+00 0.000000000000000E+00
m : 1
the solution for t :
 x : 1.000000000000000E+00  0.0
 y : 2.000000000000000E+00  0.0
== err : 0.000E+00 = rco : 1.000E+00 = res : 0.000E+00 =


In [23]:
multiplicity, hilbert_function = double_multiplicity(pols, sol)

In [24]:
multiplicity

3

Thus, the multiplicity of ``(1, 2)`` as solution equals three.  This is confirmed by the Hilbert function.

In [25]:
hilbert_function

[1, 1, 1, 0, 0, 0]